### Import

In [23]:

!pip install catboost

In [24]:
import pandas as pd
import numpy as np
import random
import os
from datetime import datetime
import sys

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso

In [25]:
from google.colab import drive
from importlib import reload

drive.mount('/content/gdrive',force_remount = True)


Mounted at /content/gdrive


### utils module import

In [26]:
if 'gdrive.MyDrive.utils' not in sys.modules:
  import  gdrive.MyDrive.utils as utils
else:
  print("utils reload")
  reload(utils)

if 'gdrive.MyDrive.data_EDA' not in sys.modules:
  import gdrive.MyDrive.data_EDA as EDA
else:
  print('eda module reload')
  reload(EDA)

utils reload
version 1
eda module reload


In [27]:
project_name = '조회수예측'

### util 설정

In [28]:
utils.seed_everything(311)
utils.set_dir(project_name)

현재 폴더 :  /content/gdrive/MyDrive/data/조회수예측
데이터 위치 :  /content/gdrive/MyDrive/data/조회수예측/sample_data
submission 위치 :  /content/gdrive/MyDrive/data/조회수예측/sample_data


### Load Data

In [ ]:
train, test, submission = utils.get_data()
train.head(5)


load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv
load from submission :  /content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv


,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,1.0,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,1.0,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,1.0,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000


### 2-1 데이터 전처리1. 정규화
new : 정규화 x  
quality : standard  
duration : standard  
bounced : standard  
trainsaction : standard  
transcation_revenue : standard  
   

In [29]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler
minmaxsc = MinMaxScaler()
stdsc = StandardScaler()


In [ ]:
train[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(train[['quality','duration','transaction','transaction_revenue']])
test[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(test[['quality','duration','transaction','transaction_revenue']])

### 2-2 데이터 전처리2. 결측치 채우기
결측치는 전부 string타입이므로 ""로 채운다

In [ ]:
train.fillna("",inplace=True)
test.fillna("",inplace=True)

### 2-3 데이터전처리3. ID 지우기


In [ ]:
train.drop(columns=['sessionID','userID'], inplace=True)
test.drop(columns=['sessionID','userID'],inplace = True)
train.head(5)

,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,17.0,Chrome,Macintosh,desktop,0,3.713206,2.193865,0,-0.089494,-0.042587,Americas,Northern America,United States,google,organic,Category8,
1,3.0,Chrome,Windows,desktop,1,-0.253972,-0.223964,0,-0.089494,-0.042587,Europe,Western Europe,Germany,google,organic,Category8,
2,1.0,Samsung Internet,Android,mobile,1,-0.253972,-0.341833,1,-0.089494,-0.042587,Asia,Southeast Asia,Malaysia,(direct),(none),,
3,1.0,Chrome,Macintosh,desktop,1,-0.253972,-0.341833,1,-0.089494,-0.042587,Americas,Northern America,United States,Partners,affiliate,,
4,1.0,Chrome,iOS,mobile,0,-0.253972,-0.341833,1,-0.089494,-0.042587,Americas,Northern America,United States,groups.google.com,referral,,Category6_Path_0000


### 2-4 데이터전처리4. Category 변환


In [ ]:
# categorical_features = list(train.dtypes[train.dtypes == "object"].index)

# for i in categorical_features:
#     count = train[i].nunique()
#     print(f"{i}: {count}")
# encoding_target = list(train.dtypes[train.dtypes == "object"].index)
# for i in encoding_target:
#     le = LabelEncoder()
#     le.fit(train[i])
#     train[i] = le.transform(train[i])

#     # test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가
#     # test 데이터에 대해서 직접적으로 fit을 수행할 경우 Data Leakage
#     for cases in np.unique(test[i]):
#         if cases not in le.classes_:
#             le.classes_ = np.append(le.classes_, cases)

#     test[i] = le.transform(test[i])

### 2-5 데이터전처리5. 학습용 데이터 분리

In [ ]:
train_x = train.drop(columns=['TARGET'])
train_y = train['TARGET']

test_x = test

### 3-1 모델학습 2
데이터 정규화, 결측치 ""로 채우기, ID 삭제  
public  
2.96014115  
private  
2.8912050993  

In [30]:
### 3-1 모델 학습
from catboost import CatBoostRegressor, Pool

In [ ]:
cat_model = CatBoostRegressor(learning_rate=0.05,depth=8, iterations=500, verbose=5)

In [ ]:
train_pool = Pool(train_x, train_y, cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium','keyword','referral_path'])
test_pool = Pool(test_x,cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium','keyword','referral_path'])

In [ ]:
cat_model.fit(train_pool)

0:	learn: 5.3318528	total: 851ms	remaining: 7m 4s
5:	learn: 4.5905039	total: 4.62s	remaining: 6m 20s
10:	learn: 4.0601575	total: 9.71s	remaining: 7m 11s
15:	learn: 3.6817503	total: 13.1s	remaining: 6m 35s
20:	learn: 3.4171027	total: 17.3s	remaining: 6m 34s
25:	learn: 3.2349937	total: 23.7s	remaining: 7m 12s
30:	learn: 3.1054624	total: 27.9s	remaining: 7m 2s
35:	learn: 3.0161105	total: 31.3s	remaining: 6m 43s
40:	learn: 2.9478635	total: 36s	remaining: 6m 43s
45:	learn: 2.8985171	total: 39.6s	remaining: 6m 31s
50:	learn: 2.8580439	total: 42.8s	remaining: 6m 16s
55:	learn: 2.8295555	total: 46s	remaining: 6m 4s
60:	learn: 2.8057530	total: 51s	remaining: 6m 7s
65:	learn: 2.7849842	total: 54.5s	remaining: 5m 58s
70:	learn: 2.7661608	total: 57.9s	remaining: 5m 49s
75:	learn: 2.7503572	total: 1m 1s	remaining: 5m 40s
80:	learn: 2.7390650	total: 1m 5s	remaining: 5m 39s
85:	learn: 2.7285616	total: 1m 8s	remaining: 5m 31s
90:	learn: 2.7181487	total: 1m 12s	remaining: 5m 24s
95:	learn: 2.7125741	to

In [ ]:

preds = cat_model.predict(test_pool)
submission = pd.read_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv')
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_2.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,24.477354
1,SESSION_252290,1.043986
2,SESSION_252291,2.717184
3,SESSION_252292,4.230648
4,SESSION_252293,10.750848
5,SESSION_252294,3.975188
6,SESSION_252295,11.262716
7,SESSION_252296,3.654636
8,SESSION_252297,0.919373
9,SESSION_252298,0.993737


### 모델학습1
정규화x ID 삭제  
public  
2.9646579184  
private  
2.9040197222  

In [ ]:
train,test = utils.get_data()

load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv


In [ ]:
train.fillna("",inplace=True)
test.fillna("",inplace=True)
train.drop(columns=['sessionID','userID'], inplace=True)
test.drop(columns=['sessionID','userID'],inplace = True)
train_x = train.drop(columns=['TARGET'])
train_y = train['TARGET']
test_x = test

In [ ]:
train_pool = Pool(train_x, train_y, cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium','keyword','referral_path'])
test_pool = Pool(test_x,cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium','keyword','referral_path'])

In [ ]:
cat_model = CatBoostRegressor(learning_rate=0.05,depth=8, iterations=500, verbose=20)

In [ ]:
cat_model.fit(train_pool)

0:	learn: 5.3318528	total: 1.21s	remaining: 10m 3s
20:	learn: 3.4171027	total: 17.5s	remaining: 6m 40s
40:	learn: 2.9478635	total: 34.5s	remaining: 6m 26s
60:	learn: 2.8057530	total: 49.2s	remaining: 5m 53s
80:	learn: 2.7390650	total: 1m 4s	remaining: 5m 31s
100:	learn: 2.7039836	total: 1m 17s	remaining: 5m 7s
120:	learn: 2.6802950	total: 1m 31s	remaining: 4m 46s
140:	learn: 2.6580693	total: 1m 43s	remaining: 4m 24s
160:	learn: 2.6386549	total: 1m 57s	remaining: 4m 7s
180:	learn: 2.6183918	total: 2m 11s	remaining: 3m 51s
200:	learn: 2.6007688	total: 2m 24s	remaining: 3m 35s
220:	learn: 2.5814995	total: 2m 38s	remaining: 3m 19s
240:	learn: 2.5693230	total: 2m 51s	remaining: 3m 4s
260:	learn: 2.5539287	total: 3m 5s	remaining: 2m 49s
280:	learn: 2.5406604	total: 3m 18s	remaining: 2m 35s
300:	learn: 2.5274383	total: 3m 32s	remaining: 2m 20s
320:	learn: 2.5192948	total: 3m 45s	remaining: 2m 5s
340:	learn: 2.5096373	total: 3m 59s	remaining: 1m 51s
360:	learn: 2.5017631	total: 4m 13s	remainin

In [ ]:
preds = cat_model.predict(test_pool)
submission = pd.read_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv')
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_base.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,24.479663
1,SESSION_252290,1.043986
2,SESSION_252291,2.798210
3,SESSION_252292,4.276955
4,SESSION_252293,10.444992
5,SESSION_252294,4.016181
6,SESSION_252295,11.269613
7,SESSION_252296,3.792267
8,SESSION_252297,0.919373
9,SESSION_252298,0.993737


### 모델학습 3. 결측치 column 제거

In [ ]:
train, test, submission = utils.get_data()
train.drop(columns=['sessionID','userID', 'keyword','referral_path'], inplace=True)
test.drop(columns=['sessionID','userID', 'keyword','referral_path'], inplace=True)
train[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(train[['quality','duration','transaction','transaction_revenue']])
test[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(test[['quality','duration','transaction','transaction_revenue']])
train_x = train.drop(columns=['TARGET'])
train_y = train['TARGET']
test_x = test

train_pool = Pool(train_x, train_y, cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium'])
test_pool = Pool(test_x,cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium'])

load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv
load from submission :  /content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv


In [ ]:
cat_model = CatBoostRegressor(learning_rate=0.05,depth=8, iterations=500, verbose=20)

In [ ]:
cat_model.fit(train_pool)

0:	learn: 5.3308346	total: 626ms	remaining: 5m 12s
20:	learn: 3.4222959	total: 14.4s	remaining: 5m 27s
40:	learn: 2.9481869	total: 27.6s	remaining: 5m 9s
60:	learn: 2.8075552	total: 39.5s	remaining: 4m 44s
80:	learn: 2.7465588	total: 50.2s	remaining: 4m 19s
100:	learn: 2.7110984	total: 1m	remaining: 3m 57s
120:	learn: 2.6848007	total: 1m 10s	remaining: 3m 42s
140:	learn: 2.6584409	total: 1m 23s	remaining: 3m 32s
160:	learn: 2.6331134	total: 1m 34s	remaining: 3m 19s
180:	learn: 2.6169363	total: 1m 42s	remaining: 3m 1s
200:	learn: 2.5969897	total: 1m 54s	remaining: 2m 50s
220:	learn: 2.5819525	total: 2m 5s	remaining: 2m 38s
240:	learn: 2.5666948	total: 2m 16s	remaining: 2m 26s
260:	learn: 2.5542388	total: 2m 25s	remaining: 2m 13s
280:	learn: 2.5437043	total: 2m 35s	remaining: 2m 1s
300:	learn: 2.5318007	total: 2m 47s	remaining: 1m 50s
320:	learn: 2.5231889	total: 2m 58s	remaining: 1m 39s
340:	learn: 2.5107766	total: 3m 10s	remaining: 1m 28s
360:	learn: 2.5024601	total: 3m 21s	remaining: 

In [ ]:
preds = cat_model.predict(test_pool)
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_3.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,23.426959
1,SESSION_252290,1.002113
2,SESSION_252291,2.726032
3,SESSION_252292,4.254198
4,SESSION_252293,11.147082
5,SESSION_252294,3.987691
6,SESSION_252295,10.509360
7,SESSION_252296,3.534509
8,SESSION_252297,0.943558
9,SESSION_252298,1.016760


### 모델학습 4. randomizedSeach

In [ ]:
train, test, submission = utils.get_data()
train.drop(columns=['sessionID','userID'], inplace=True)
test.drop(columns=['sessionID','userID'], inplace=True)
train[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(train[['quality','duration','transaction','transaction_revenue']])
test[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(test[['quality','duration','transaction','transaction_revenue']])
train.fillna("",inplace=True)
test.fillna("",inplace=True)
train_x = train.drop(columns=['TARGET'])
train_y = train['TARGET']
test_x = test

train_pool = Pool(train_x, train_y, cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium', 'keyword','referral_path'])
test_pool = Pool(test_x,cat_features = ['new','browser','OS','device','bounced','continent','subcontinent','country','traffic_source','traffic_medium', 'keyword','referral_path'])

load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv
load from submission :  /content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv


In [ ]:
from sklearn.model_selection import RandomizedSearchCVparam = {
    'max_depth' : [6,8,10,12],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'learning_rate': [0.03,0.05, 0.1, 0.15, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']

}
cat_model = CatBoostRegressor(
        loss_function='RMSE',
        early_stopping_rounds=50,
        has_time=True,
        iterations=500,
        verbose=100)



In [ ]:
param = {
    'max_depth' : [6,8,10,12],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'learning_rate': [0.03,0.05, 0.1, 0.15, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']

}
cat_model = CatBoostRegressor(
        loss_function='RMSE',
        early_stopping_rounds=50,
        has_time=True,
        iterations=500,
        verbose=100)


In [ ]:
cat_model.randomized_search(param, X=train_pool)

feature_importance = cat_model.get_feature_importance(prettified=True)
print ('feature_importance', feature_importance)

0:	learn: 6.0214043	test: 6.2790273	best: 6.2790273 (0)	total: 1.59s	remaining: 13m 15s
100:	learn: 2.1268284	test: 3.0909927	best: 3.0909927 (100)	total: 2m 11s	remaining: 8m 41s
200:	learn: 1.8928834	test: 3.0845948	best: 3.0822924 (186)	total: 4m 30s	remaining: 6m 42s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3.082292396
bestIteration = 186

0:	loss: 3.0822924	best: 3.0822924 (0)	total: 5m 21s	remaining: 48m 9s
0:	learn: 6.2776251	test: 6.5253874	best: 6.5253874 (0)	total: 156ms	remaining: 1m 17s
100:	learn: 2.5013652	test: 3.1264018	best: 3.1261800 (99)	total: 12.1s	remaining: 47.8s
200:	learn: 2.4053392	test: 3.1007889	best: 3.1007143 (197)	total: 21s	remaining: 31.3s
300:	learn: 2.3510877	test: 3.0977870	best: 3.0973110 (294)	total: 30.8s	remaining: 20.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3.097311002
bestIteration = 294

1:	loss: 3.0973110	best: 3.0822924 (0)	total: 5m 55s	remaining: 23m 42s
0:	learn: 6.0214449	test: 6.28243

In [ ]:
preds = cat_model.predict(test_pool)
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_4.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,32.211336
1,SESSION_252290,0.943794
2,SESSION_252291,2.629619
3,SESSION_252292,4.271175
4,SESSION_252293,10.752405
5,SESSION_252294,4.062095
6,SESSION_252295,12.094208
7,SESSION_252296,3.843749
8,SESSION_252297,0.871487
9,SESSION_252298,0.952811


In [ ]:
train, test, submission = utils.get_data()
train.drop(columns=['sessionID','userID'], inplace=True)
test.drop(columns=['sessionID','userID'], inplace=True)
train[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(train[['quality','duration','transaction','transaction_revenue']])
test[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(test[['quality','duration','transaction','transaction_revenue']])
drops = ['bounced','keyword','transaction','continent','transaction_revenue','country']
train.fillna("",inplace=True)
test.fillna("",inplace=True)
train_x = train.drop(columns=['TARGET']+drops)
train_y = train['TARGET']
test_x = test.drop(columns=drops)
features = [col for col in train_x.columns if train_x[col].dtype=='object']
print(features)
train_pool = Pool(train_x, train_y, cat_features = features)
test_pool = Pool(test_x,cat_features = features)

load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv
load from submission :  /content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv
['browser', 'OS', 'device', 'subcontinent', 'traffic_source', 'traffic_medium', 'referral_path']


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param = {
    'max_depth' : [6,8,10,12],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'learning_rate': [0.03,0.05, 0.1, 0.15, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']

}
cat_model = CatBoostRegressor(
        loss_function='RMSE',
        early_stopping_rounds=50,
        has_time=True,
        iterations=500,
        verbose=100)

In [ ]:
param = {
    'max_depth' : [6,8,10,12],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'learning_rate': [0.03,0.05, 0.1, 0.15, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']

}
cat_model = CatBoostRegressor(
        loss_function='RMSE',
        early_stopping_rounds=20,
        has_time=True,
        iterations=500,
        verbose=100)

In [ ]:
cat_model.randomized_search(param, X=train_pool)

feature_importance = cat_model.get_feature_importance(prettified=True)
print ('feature_importance', feature_importance)

0:	learn: 6.0130708	test: 6.2702532	best: 6.2702532 (0)	total: 706ms	remaining: 5m 52s
100:	learn: 2.0909910	test: 3.1183631	best: 3.1182700 (97)	total: 1m 31s	remaining: 6m
Stopped by overfitting detector  (20 iterations wait)

bestTest = 3.116496875
bestIteration = 119

0:	loss: 3.1164969	best: 3.1164969 (0)	total: 2m 4s	remaining: 18m 39s
0:	learn: 6.2737985	test: 6.5207144	best: 6.5207144 (0)	total: 139ms	remaining: 1m 9s
100:	learn: 2.4786410	test: 3.1428483	best: 3.1428483 (100)	total: 11s	remaining: 43.4s
200:	learn: 2.3633131	test: 3.1224284	best: 3.1216417 (198)	total: 20.5s	remaining: 30.5s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 3.115006577
bestIteration = 275

1:	loss: 3.1150066	best: 3.1150066 (1)	total: 2m 31s	remaining: 10m 7s
0:	learn: 6.0181935	test: 6.2746112	best: 6.2746112 (0)	total: 178ms	remaining: 1m 28s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 3.110822287
bestIteration = 59

2:	loss: 3.1108223	best: 3.1108223 (2

In [ ]:
preds = cat_model.predict(test_pool)
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_5.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,29.193310
1,SESSION_252290,1.004343
2,SESSION_252291,2.755395
3,SESSION_252292,4.254916
4,SESSION_252293,11.419766
5,SESSION_252294,3.957870
6,SESSION_252295,11.198855
7,SESSION_252296,3.681704
8,SESSION_252297,0.997692
9,SESSION_252298,0.998682


In [35]:
train, test, submission = utils.get_data()
train.drop(columns=['sessionID','userID'], inplace=True)
test.drop(columns=['sessionID','userID'], inplace=True)
train[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(train[['quality','duration','transaction','transaction_revenue']])
test[['quality','duration','transaction','transaction_revenue']] = stdsc.fit_transform(test[['quality','duration','transaction','transaction_revenue']])
drops = ['bounced','keyword','continent','transaction','browser']
train.fillna("",inplace=True)
test.fillna("",inplace=True)
train_x = train.drop(columns=['TARGET']+drops)
train_y = train['TARGET']
test_x = test.drop(columns=drops)
features = [col for col in train_x.columns if train_x[col].dtype=='object']
print(features)
train_pool = Pool(train_x, train_y, cat_features = features)
test_pool = Pool(test_x,cat_features = features)

load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/train.csv
load from train :  /content/gdrive/MyDrive/data/조회수예측/sample_data/test.csv
load from submission :  /content/gdrive/MyDrive/data/조회수예측/sample_data/sample_submission.csv
['OS', 'device', 'subcontinent', 'country', 'traffic_source', 'traffic_medium', 'referral_path']


In [36]:
from sklearn.model_selection import RandomizedSearchCV
param = {
    'max_depth' : [6,8,10,12],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'learning_rate': [0.03,0.05, 0.1, 0.15, 0.2],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']

}
cat_model = CatBoostRegressor(
        loss_function='RMSE',
        early_stopping_rounds=15,
        has_time=True,
        iterations=500,
        verbose=100)

In [ ]:
cat_model.randomized_search(param, X=train_pool)

feature_importance = cat_model.get_feature_importance(prettified=True)
print ('feature_importance', feature_importance)

0:	learn: 6.0201179	test: 6.2850028	best: 6.2850028 (0)	total: 1.29s	remaining: 10m 44s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 3.10294561
bestIteration = 65

0:	loss: 3.1029456	best: 3.1029456 (0)	total: 1m 12s	remaining: 10m 49s
0:	learn: 6.2770341	test: 6.5248933	best: 6.5248933 (0)	total: 286ms	remaining: 2m 22s
100:	learn: 2.4934312	test: 3.1388067	best: 3.1388067 (100)	total: 11s	remaining: 43.4s
200:	learn: 2.3820664	test: 3.1114991	best: 3.1113751 (190)	total: 22.2s	remaining: 33s
300:	learn: 2.3230684	test: 3.1023754	best: 3.1021428 (297)	total: 31.1s	remaining: 20.6s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 3.100963026
bestIteration = 326

1:	loss: 3.1009630	best: 3.1009630 (1)	total: 1m 46s	remaining: 7m 5s
0:	learn: 6.0248863	test: 6.2845112	best: 6.2845112 (0)	total: 198ms	remaining: 1m 38s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 3.093615166
bestIteration = 76

2:	loss: 3.0936152	best: 3.0936152 (

In [ ]:
preds = cat_model.predict(test_pool)
submission['TARGET'] = preds
display(submission.head(10))
submission.to_csv('/content/gdrive/MyDrive/data/조회수예측/sample_data/cat_10.csv', index=False)

,sessionID,TARGET
0,SESSION_252289,35.446317
1,SESSION_252290,1.008764
2,SESSION_252291,2.553278
3,SESSION_252292,4.208035
4,SESSION_252293,10.117527
5,SESSION_252294,3.874343
6,SESSION_252295,11.513831
7,SESSION_252296,3.622024
8,SESSION_252297,0.995114
9,SESSION_252298,0.986281
